In [27]:
import os
from dotenv import load_dotenv
from typing import TypedDict, Sequence, List, Literal, Annotated, cast
from pydantic import BaseModel, Field

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser
from langchain_groq.chat_models import ChatGroq
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
from langchain_core.tools.retriever import create_retriever_tool
from langchain_community.tools.tavily_search import TavilySearchResults

from  langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages, MessagesState, BaseMessage
from langgraph.prebuilt import tools_condition, ToolNode

In [2]:
groq_api_key = os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

llm = ChatGroq(groq_api_key=groq_api_key, model="gemma2-9b-it")
llm.invoke("hi").content

'Hello! 👋 \n\nHow can I help you today? 😊\n'

In [3]:
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [4]:
file_path = r"E:\2025\Generative_AI\LangGraph\LangGraph_E2E\data\Gujrat_accident.pdf"

docs = PyPDFLoader(file_path).load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = text_splitter.split_documents(docs)

vectordb = Chroma.from_documents(documents=documents,
                                 collection_name="rag",
                                 embedding=embeddings)

retriever = vectordb.as_retriever()

In [5]:
retriever.get_relevant_documents("main cause of air crash?")

C:\Users\sowmy\AppData\Local\Temp\ipykernel_17948\3813989304.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("main cause of air crash?")


[Document(metadata={'total_pages': 3, 'creator': 'LaTeX with hyperref', 'page_label': '2', 'source': 'E:\\2025\\Generative_AI\\LangGraph\\LangGraph_E2E\\data\\Gujrat_accident.pdf', 'producer': 'xdvipdfmx (20220710)', 'page': 1, 'creationdate': '2025-06-17T17:22:18+00:00'}, page_content='cient thrust. Unverified claims on X mention fuel contamination, but no\nevidence supports this. Weather was clear (visibility 6 km, light winds 3–6\nknots), ruling out meteorological factors.\n• Investigating Bodies: Aircraft Accident Investigation Bureau (AAIB) India,\nDGCA, US NTSB, FAA, UK AAIB, and a high-level committee led by Union\nHome Secretary Govind Mohan. Final report expected by June 2026.\nFigure 3: Recovered black box of Flight AI171, critical for determining the cause\nof the crash.'),
 Document(metadata={'page': 2, 'creationdate': '2025-06-17T17:22:18+00:00', 'source': 'E:\\2025\\Generative_AI\\LangGraph\\LangGraph_E2E\\data\\Gujrat_accident.pdf', 'creator': 'LaTeX with hyperref', 'pro

In [6]:
retrieve_tool = create_retriever_tool(
    retriever=retriever,
    name="rag-tool",
    description="If the question is related to Ahamadabad air crash use this tool"
)

In [7]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

In [8]:
class Analyser(BaseModel):
    Topic: str = Field(description="Selected topic 'Retriever call', 'LLM call', 'Internet call'")
    Reason: str = Field(description="Reason for selected topic")

In [9]:
def supervisor(state: AgentState):
    print("--- SUPERVISOR ---")
    question = state['messages'][0]
    
    parser = PydanticOutputParser(pydantic_object=Analyser)
    
    template = """You are an ai assistant tasked with identify proper topic based on the question.
    1. Retriever call: If the question is related to Ahamadabad air crash
    2. LLM call: If the question is related to old news
    3. Internet call: If the question is related to current news
    Note: Provide only these topics 'Retriever call', 'LLM call', 'Internet call'
    Question: {question}\n{format_instructions}
    """
    
    prompt = PromptTemplate(
        template=template,
        input_variables=["question"],
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )
    
    chain = prompt | llm | parser
    
    response = chain.invoke({"question": question})
    print(response.Topic)
    return {"messages": [response.Topic]}

In [10]:
supervisor({"messages": ["what is the cause of air crash"]})

--- SUPERVISOR ---
Retriever call


{'messages': ['Retriever call']}

In [11]:
def router(state: AgentState):
    print("--- ROUTER ---")
    last_message = state['messages'][-1].content
    print(last_message)
    
    if "Retriever call" in last_message:
        return "RAG"
    elif "LLM call" in last_message:
        return "LLM"
    elif "Internet call" in last_message:
        return "Internet"

In [12]:
def retrieve(state: AgentState):
    question = state['messages'][0]
    return {"messages": [retrieve_tool.invoke(question)]}

In [13]:
retrieve({"messages": ["cause of air crash"]})

{'messages': ['cient thrust. Unverified claims on X mention fuel contamination, but no\nevidence supports this. Weather was clear (visibility 6 km, light winds 3–6\nknots), ruling out meteorological factors.\n• Investigating Bodies: Aircraft Accident Investigation Bureau (AAIB) India,\nDGCA, US NTSB, FAA, UK AAIB, and a high-level committee led by Union\nHome Secretary Govind Mohan. Final report expected by June 2026.\nFigure 3: Recovered black box of Flight AI171, critical for determining the cause\nof the crash.\n\ncrash site in Ahmedabad, June 12, 2025.\n7 Conclusion\nThe Air India Flight AI171 crash is India’s deadliest aviation disaster since 1996\nand the first fatal incident involving a Boeing 787 Dreamliner. The ongoing in-\nvestigation, supported by the black box, aims to uncover the cause, with engine\nfailure as a leading hypothesis. The tragedy has prompted global condolences\nfrom leaders like PM Narendra Modi, UK PM Keir Starmer, and US President\nDonald Trump.\nReference

In [14]:
class AnswerParser(BaseModel):
    Answer: str = Field(description="The final answer to the user question")
    Source: str = Field(description="Origin of this answer (RAG, LLM, Internet)")
    
answer_parser = PydanticOutputParser(pydantic_object=AnswerParser)

In [15]:
def call_llm(state: AgentState):
    print("--- CALL LLM ---")
    # question = state['messages']
        
    # template = """You are a helpful assistant answer the following question with accurate 
    # Question: {question}
    # {format_instructions}
    # """
    
    # prompt = PromptTemplate(
    #     template=template,
    #     input_variables=["question"],
    #     partial_variables={"format_instructions": answer_parser.get_format_instructions()}
    # )
    
    # chain = prompt | llm | answer_parser
    
    # return {"messages": [chain.invoke(question)]}
    
    question = state['messages'][0]
    print(question)
    
    llm_with_structured_op = llm.with_structured_output(AnswerParser)
    
    return {"messages": [llm_with_structured_op.invoke(question)]}

In [16]:
call_llm({"messages": ["first prime minister of india?"]})

--- CALL LLM ---
first prime minister of india?


{'messages': [AnswerParser(Answer='Jawaharlal Nehru', Source='LLM')]}

In [17]:
def call_interner(state: AgentState):
    print("--- CALL INTERNET ---")
    question = state['messages'][0]
    print(question)
    
    tavily_tool = TavilySearchResults()
    response = tavily_tool.invoke(question.content)
    print(response)
    
    return {"messages": [response]}

In [26]:
tavily_tool = TavilySearchResults()
tavily_tool.invoke("president of inida?")

[{'url': 'https://en.wikipedia.org/wiki/President_of_India',
  'content': 'The **president of India** ([ISO](https://en.wikipedia.org/wiki/ISO_15919 "ISO 15919"): _Bhārata kē Rāṣṭrapati_) is the [head of state](https://en.wikipedia.org/wiki/Head_of_state "Head of state") of the [Republic of India](https://en.wikipedia.org/wiki/Republic_of_India "Republic of India"). The president is the nominal head of the executive,[[a]](https://en.wikipedia.org/wiki/President_of_India#cite_note-3) the first citizen of the country, and the [supreme [...] *   **President of India**\n\n*   [Droupadi Murmu](https://en.wikipedia.org/wiki/Droupadi_Murmu "Droupadi Murmu") ([15th](https://en.wikipedia.org/wiki/List_of_Presidents_of_India "List of Presidents of India"))\n\n* * *\n\n*   **[Vice President of India](https://en.wikipedia.org/wiki/Vice_President_of_India "Vice President of India")** [...] | President of India |\n| --- |\n| _Bhārata kē Rāṣṭrapati_ |\n| [![Image 5](https://upload.wikimedia.org/wikip

In [18]:
def validator(state: AgentState):
    print("--- VALIDATOR ---")
    last_message = state['messages'][-1]
    
    parser = PydanticOutputParser(pydantic_object=AnswerParser)
    
    try:
        validated = parser.parse(last_message.content) 
        print("VALIDATED: ", validated)
        print("Validation PASSED")
        return{"messages": ['Success']}
    except Exception as e:
        print("Validation FAILED", e)
        return{"messages": ['Supervisor']}
        

In [19]:
workflow = StateGraph(AgentState)

workflow.add_node("Supervisor", supervisor)
# workflow.add_node("RAG", retrieve)
workflow.add_node("RAG", ToolNode([retrieve_tool]))
workflow.add_node("LLM", call_llm)
workflow.add_node("Internet", call_interner)
# workflow.add_node("Validator", validator)

workflow.add_edge(START, "Supervisor")
workflow.add_conditional_edges(
    "Supervisor",
    router,
    {
        "RAG": "RAG",
        "LLM": "LLM",
        "Internet": "Internet"
    }
)

workflow.add_edge("RAG", END)
workflow.add_edge("LLM", END)
workflow.add_edge("Internet", END)

# workflow.add_edge("RAG", "Validator")
# workflow.add_edge("LLM", "Validator")
# workflow.add_edge("Internet", "Validator")

# workflow.add_conditional_edges(
#     validator,
#     lambda state: "Supervisor" if not state['valid'] else "Feedback",
#     {
#         "Supervisor": "Supervisor",
#         "Feedback": END
#     }
# )

# workflow.add_edge("Validator", END)

app = workflow.compile()


In [20]:
from langchain_core.messages import HumanMessage

# app.invoke({"messages": ["cause for air crash"]})

app.invoke({"messages": ["current president of india?"]})

--- SUPERVISOR ---
Internet call
--- ROUTER ---
Internet call
--- CALL INTERNET ---
content='current president of india?' additional_kwargs={} response_metadata={} id='edf5965d-7e9e-4213-bfdb-22b5c124af03'
[{'url': 'https://en.wikipedia.org/wiki/President_of_India', 'content': 'Murmu](/wiki/Droupadi_Murmu "Droupadi Murmu") is the 15th and current president, having taken office on 25 July 2022. [...] | President of the Republic of India | |\n| --- | --- |\n| *Bhārata kē Rāṣṭrapati* | |\n| Logo of the President of India | |\n| [f](/wiki/File:Flag_of_India.svg) [Flag of India](/wiki/Flag_of_India "Flag of India") | |\n| Incumbent [Droupadi Murmu](/wiki/Droupadi_Murmu "Droupadi Murmu") since\xa025 July 2022 | | [...] The **president of India** ([ISO](/wiki/ISO_15919 "ISO 15919"): *Bhārata kē Rāṣṭrapati*) is the [head of state](/wiki/Head_of_state "Head of state") of the [Republic of India](/wiki/Republic_of_India "Republic of India"). The president is the nominal head of the executive,[[a]

NotImplementedError: Unsupported message type: <class 'list'>
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/MESSAGE_COERCION_FAILURE 